In [ ]:
#pip install langgraph
#pip install langchain-nomic
#pip install -U langchain-ollama
#pip install tiktoken
#pip install langchain-community
#pip install gpt4all

In [1]:

import ollama
response = ollama.pull(model="llama3.2:3b-instruct-fp16")
response
from langchain_nomic import *
import langchain_community
from langchain_ollama import ChatOllama

local_llm = "llama3.2:3b-instruct-fp16"
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json")


In [3]:
import os
import getpass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("TAVILY_API_KEY")
os.environ["TOKENIZERS_PARALLELISM"] = "true"

TAVILY_API_KEY:  ········


In [5]:
from gpt4all import GPT4All

In [161]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# Load documents
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local"),
)

# Create retriever
retriever = vectorstore.as_retriever(k=3)

In [9]:
pip -q install git+https://github.com/openai/swarm.git

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.3 requires tenacity!=8.4.0,<9.0.0,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.
langchain-community 0.3.2 requires tenacity!=8.4.0,<9.0.0,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.
streamlit 1.32.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.


In [13]:
from swarm import Swarm, Agent

In [15]:
import json

In [139]:
import json
from langchain_ollama import ChatOllama
from langchain.schema import SystemMessage, HumanMessage, AIMessage

# Swarm client using local LLM
# Define the local language model
local_llm = "llama3.2:3b-instruct-fp16"
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json")

# Swarm Client
class Swarm:
    def __init__(self, model, temperature=0):
        self.llm = ChatOllama(model=model, temperature=temperature)

    def run(self, agent, messages, context_variables=None, max_turns=float("inf")):
        context_variables = context_variables or {}
        current_agent = agent
        response_messages = messages[:]
        turn_count = 0

        while turn_count < max_turns:
            # Prepare the messages for the current agent
            messages_for_llm = self._prepare_messages(current_agent, response_messages, context_variables)

            # Generate response using the local LLM
            response = self.llm(messages_for_llm)
            
            # Add response to messages
            response_message = AIMessage(content=response.content)
            response_messages.append(response_message)
            
            # Check if agent wants to hand off to another agent
            next_agent = self._handle_handoff(current_agent, response, context_variables)
            if next_agent is None:
                break
            else:
                current_agent = next_agent

            turn_count += 1

        return {
            "messages": response_messages,
            "agent": current_agent,
            "context_variables": context_variables,
        }
        
    def _prepare_messages(self, agent, messages, context_variables):
        # Prepare the messages using agent instructions and previous messages
        instructions = agent.instructions if isinstance(agent.instructions, str) else agent.instructions(context_variables)
        system_message = SystemMessage(content=instructions)
        return [system_message] + messages

    def _handle_handoff(self, agent, response, context_variables):
        # Check if the response includes a handoff to another agent
        if "handoff" in response.content:
            # Assuming the handoff is specified in the response content
            return response.content.get("handoff")
        return None

# Define the Agent class (assuming it's not defined elsewhere)
class Agent:
    def __init__(self, name, instructions):
        self.name = name
        self.instructions = instructions

# Instantiate Swarm client with local LLM
client = Swarm(model=local_llm)

# Define Router Agent
router_agent = Agent(
    name="Router Agent",
    instructions="""You are an expert at routing a user question to a vectorstore or web search.

The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.

Use the vectorstore for questions on these topics. For all else, and especially for current events, use web-search.

Return JSON with a single key, datasource, that is 'websearch' or 'vectorstore' depending on the question.""",
)

# Test the router with questions

# Define test questions
test_questions = [
    "Who is favored to win the NFC Championship game in the 2024 season?",
]

# Loop through questions and test router
for question in test_questions:
    messages = [
        HumanMessage(content=question)
    ]
    response = client.run(agent=router_agent, messages=messages)

    # Parse the response as JSON and print the routing decision
    try:
        response_json = json.loads(response["messages"][-1].content)
        print(f"Question: {question}")
        print(f"Routing Decision: {response_json}\n")
    except json.JSONDecodeError:
        print(f"Failed to parse response for question: {question}\n")
    except ValueError:
        print(f"Received unsupported message type for Ollama for question: {question}\n")


Question: Who is favored to win the NFC Championship game in the 2024 season?
Routing Decision: {'datasource': 'websearch'}



In [55]:


# Doc grader instructions
doc_grader_instructions = """You are a grader assessing relevance of a retrieved document to a user question.

If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant."""

# Define Retrieval Grader Agent
retrieval_grader_agent = Agent(
    name="Retrieval Grader Agent",
    instructions=doc_grader_instructions
)

# Define grade_document function
def grade_document(document, question):
    """Grade the document based on relevance to the question."""
    doc_grader_prompt_formatted = f"""Here is the retrieved document: \n\n {document} \n\n Here is the user question: \n\n {question}.

Carefully and objectively assess whether the document contains at least some information that is relevant to the question.

Return JSON with a single key, binary_score, that is 'yes' or 'no' score to indicate whether the document contains at least some information that is relevant to the question."""

    result = client.llm.invoke(doc_grader_prompt_formatted)
    response_content = result.content.strip()
    if not response_content:
        raise ValueError("Received empty response from the model.")
    try:
        # Extract JSON content if the response contains additional text
        json_start_idx = response_content.find("{")
        json_end_idx = response_content.rfind("}") + 1
        json_content = response_content[json_start_idx:json_end_idx]
        return json.loads(json_content)
    except json.JSONDecodeError:
        raise ValueError(f"Failed to parse JSON from the response: {response_content}")

# Test the Retrieval Grader Agent
# Define question and retrieved document
question = "What is Chain of thought prompting?"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content

# Run agent with question and document
context_variables = {
    "document": doc_txt,
    "question": question
}

# Use the grade_document function
grade_result = grade_document(context_variables["document"], context_variables["question"])
print(f"Grading Result: {grade_result}")


Grading Result: {'binary_score': 'yes'}


In [141]:
### Generate

# Prompt
rag_prompt = """You are an assistant for question-answering tasks. 

Here is the context to use to answer the question:

{context} 

Think carefully about the above context. 

Now, review the user question:

{question}

Provide an answer to this questions using only the above context. 

Use three sentences maximum and keep the answer concise.

Answer:"""


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Test
docs = retriever.invoke(question)
docs_txt = format_docs(docs)
rag_prompt_formatted = rag_prompt.format(context=docs_txt, question=question)
generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
print(generation.content)

I cannot provide real-time sports updates or predictions. However, I can tell you that the NFC Championship game is a significant event in the NFL season, and teams are often favored to win based on their performance and strength of schedule. Without access to current team rankings, recent performances, and other relevant factors, it's difficult to make an accurate prediction about which team will emerge victorious in the 2024 NFC Championship game.


In [79]:
# Hallucination grader instructions
hallucination_grader_instructions = """

You are a teacher grading a quiz. 

You will be given FACTS and a STUDENT ANSWER. 

Here is the grade criteria to follow:

(1) Ensure the STUDENT ANSWER is grounded in the FACTS. 

(2) Ensure the STUDENT ANSWER does not contain "hallucinated" information outside the scope of the FACTS.

Score:

A score of yes means that the student's answer meets all of the criteria. This is the highest (best) score. 

A score of no means that the student's answer does not meet all of the criteria. This is the lowest possible score you can give.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Define Hallucination Grader Agent
hallucination_grader_agent = Agent(
    name="Hallucination Grader Agent",
    instructions=hallucination_grader_instructions
)

# Define grade_hallucination function
def grade_hallucination(documents, answer):
    """Grade the student's answer based on the given facts."""
    hallucination_grader_prompt_formatted = f"""FACTS: \n\n {documents} \n\n STUDENT ANSWER: {answer}. 

Return JSON with two keys: binary_score is 'yes' or 'no' score to indicate whether the STUDENT ANSWER is grounded in the FACTS. And a key, explanation, that contains an explanation of the score."""

    result = client.llm.invoke(hallucination_grader_prompt_formatted)
    response_content = result.content.strip()
    if not response_content:
        raise ValueError("Received empty response from the model.")
    try:
        # Extract JSON content if the response contains additional text
        json_start_idx = response_content.find("{")
        json_end_idx = response_content.rfind("}") + 1
        json_content = response_content[json_start_idx:json_end_idx]
        return json.loads(json_content)
    except json.JSONDecodeError:
        raise ValueError(f"Failed to parse JSON from the response: {response_content}")

# Test the Hallucination Grader Agent
# Define facts and student answer
facts = "The Llama 3.2 models include 11B Vision Instruct and 90B Vision Instruct models, available on Azure AI Model Catalog."
answer = "The Llama 3.2 models released today include two vision models: Llama 3.2 11B Vision Instruct and Llama 3.2 90B Vision Instruct, available on Azure AI Model Catalog."

# Run agent with facts and answer
context_variables = {
    "documents": facts,
    "answer": answer
}

# Use the grade_hallucination function
grade_result = grade_hallucination(context_variables["documents"], context_variables["answer"])
print(f"Grading Result: {grade_result}")


Grading Result: {'binary_score': 'no', 'explanation': "The STUDENT ANSWER incorrectly states that the Llama 3.2 models were released 'today'. The FACTS do not provide a specific release date."}


In [142]:

# Answer grader instructions
answer_grader_instructions = """You are a teacher grading a quiz. 

You will be given a QUESTION and a STUDENT ANSWER. 

Here is the grade criteria to follow:

(1) The STUDENT ANSWER helps to answer the QUESTION

Score:

A score of yes means that the student's answer meets all of the criteria. This is the highest (best) score. 

The student can receive a score of yes if the answer contains extra information that is not explicitly asked for in the question.

A score of no means that the student's answer does not meet all of the criteria. This is the lowest possible score you can give.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Define Answer Grader Agent
answer_grader_agent = Agent(
    name="Answer Grader Agent",
    instructions=answer_grader_instructions
)

# Define grade_answer function
def grade_answer(question, answer):
    """Grade the student's answer based on the given question."""
    answer_grader_prompt_formatted = f"""QUESTION: \n\n {question} \n\n STUDENT ANSWER: {answer}. 

Return JSON with two keys: binary_score is 'yes' or 'no' score to indicate whether the STUDENT ANSWER meets the criteria. And a key, explanation, that contains an explanation of the score."""

    result = client.llm.invoke(answer_grader_prompt_formatted)
    response_content = result.content.strip()
    if not response_content:
        raise ValueError("Received empty response from the model.")
    try:
        # Extract JSON content if the response contains additional text
        json_start_idx = response_content.find("{")
        json_end_idx = response_content.rfind("}") + 1
        json_content = response_content[json_start_idx:json_end_idx]
        return json.loads(json_content)
    except json.JSONDecodeError:
        raise ValueError(f"Failed to parse JSON from the response: {response_content}")

# Test the Answer Grader Agent
# Define question and student answer
question = "What are the vision models released today as part of Llama 3.2?"
answer = "The Llama 3.2 models released today include two vision models: Llama 3.2 11B Vision Instruct and Llama 3.2 90B Vision Instruct, which are available on Azure AI Model Catalog via managed compute. These models are part of Meta's first foray into multimodal AI and rival closed models like Anthropic's Claude 3 Haiku and OpenAI's GPT-4o mini in visual reasoning. They replace the older text-only Llama 3.1 models."

# Run agent with question and answer
context_variables = {
    "question": question,
    "answer": answer
}

# Use the grade_answer function
grade_result = grade_answer(context_variables["question"], context_variables["answer"])
print(f"Grading Result: {grade_result}")


Grading Result: {'binary_score': 'no', 'explanation': 'The student answer does not provide accurate information about the vision models released as part of Llama 3.2. The correct models are Llama 3.2 11B Vision Instruct and Llama 3.2 90B Vision Instruct, which are text-to-image models, not visual reasoning models like Claude 3 Haiku or GPT-4o mini.'}


In [61]:
# Define Web Search Agent without Tavily
web_search_instructions = """You are an expert search engine. Retrieve the top 3 relevant search results based on the user's query."""

web_search_agent = Agent(
    name="Web Search Agent",
    instructions=web_search_instructions
)

# Define search_web function
def search_web(query):
    """Perform a web search based on the given query."""
    search_prompt_formatted = f"Search query: {query}\n\nReturn the top 3 search results as a list of URLs." 

    result = client.llm.invoke(search_prompt_formatted)
    response_content = result.content.strip()
    if not response_content:
        raise ValueError("Received empty response from the model.")
    return response_content.splitlines()

# Test the Web Search Agent
search_query = "What are the benefits of Chain of Thought prompting?"
search_results = search_web(search_query)
print(f"Search Results: {search_results}")

Search Results: ["I'm happy to help you with your question, but I need to clarify that I'm a large language model, I don't have direct access to search engines like Google. However, I can provide you with some general information about Chain of Thought prompting and its benefits.", '', 'Chain of Thought (CoT) prompting is a technique used in natural language processing (NLP) and artificial intelligence (AI) that involves generating text by following a logical sequence of thoughts or ideas. Here are three potential benefits of using CoT prompting:', '', '1. Improved coherence and fluency: CoT prompting can help generate more coherent and fluent text by ensuring that the generated text follows a logical sequence of thoughts.', '2. Enhanced creativity: By allowing the model to explore different branches of thought, CoT prompting can enable the generation of more creative and innovative ideas.', '3. Better understanding of complex topics: CoT prompting can help models better understand com

In [145]:
### Search
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

# Define search_web function non Agentic
def search_web(query):
    """Perform a web search based on the given query."""
    web_search_tool = TavilySearchResults(k=3)
    search_results = web_search_tool.invoke(query)
    
    # Extract links or summarize content from search results
    results = []
    for result in search_results:
        if "link" in result:
            results.append(result["link"])
        elif "content" in result:
            results.append(result["content"])
        else:
            results.append("No link or content available")
    
    return results

# Test the Web Search Agent
search_query = "What are the benefits of Chain of Thought prompting?"
search_results = search_web(search_query)
print(f"Search Results: {search_results}")


In [147]:
# Define Web Search Agent
web_search_instructions = """You are an expert search engine. Retrieve the top 3 relevant search results based on the user's query."""

web_search_agent = Agent(
    name="Web Search Agent",
    instructions=web_search_instructions
)

# Define search_web function
def search_web(query):
    """Perform a web search based on the given query."""
    web_search_tool = TavilySearchResults(k=3)
    search_results = web_search_tool.invoke(query)
    return [result["content"] for result in search_results]

# Test the Web Search Agent
search_query = "What are the benefits of Chain of Thought prompting?"
search_results = search_web(search_query)
print(f"Search Results: {search_results}")


Search Results: ['Benefits of Chain-of-Thought Prompting. Chain-of-Thought prompting offers several benefits, particularly in enhancing the performance and reliability of language models in complex tasks. #1. Improved accuracy. By breaking down a problem into smaller, manageable steps, the model can handle complex tasks more accurately.', "Chain-of-Thought prompting offers several significant benefits in advancing the reasoning capabilities of large language models. Let's explore some of the key advantages: Improved Performance on Complex Reasoning Tasks. One of the primary benefits of CoT prompting is its ability to enhance the performance of language models on complex", 'Another substantial benefit of the chain of thought prompting is its interpretability. One of the inherent challenges with large language models is the so-called "black box" nature of their responses. In traditional methods, the LLM generates an output, but the pathway it took to reach that conclusion often remains o

In [63]:
import operator
from typing_extensions import TypedDict
from typing import List, Annotated


class GraphState(TypedDict):
    """
    Graph state is a dictionary that contains information we want to propagate to, and modify in, each graph node.
    """

    question: str  # User question
    generation: str  # LLM generation
    web_search: str  # Binary decision to run web search
    max_retries: int  # Max number of retries for answer generation
    answers: int  # Number of answers generated
    loop_step: Annotated[int, operator.add]
    documents: List[str]  # List of retrieved documents


In [71]:
# Combined process_question function to use all agents
def process_question(question):
    """Process the question and return the final answer."""
    # Retrieve documents using retriever
    docs = retriever.invoke(question)
    doc_txt = docs[1].page_content if docs else None

    if doc_txt:
        # Use Retrieval Grader Agent to grade the retrieved document
        grade_result = grade_document(doc_txt, question)
        if grade_result.get("binary_score") == "yes":
            print(f"Retrieved document is relevant: {grade_result}")
        else:
            # If document is not relevant, use Web Search Agent
            print("Document not relevant, performing web search...")
            search_results = search_web(question)
            print(f"Search Results: {search_results}")
    else:
        # If no documents are retrieved, perform web search
        print("No documents retrieved, performing web search...")
        search_results = search_web(question)
        print(f"Search Results: {search_results}")

    # Use Answer Grader Agent to grade a sample answer (for demonstration)
    sample_answer = "This is a sample answer based on the retrieved or searched information."
    answer_grade_result = grade_answer(question, sample_answer)
    print(f"Answer Grading Result: {answer_grade_result}")

# Example usage
process_question("What are the types of agent memory?")

Retrieved document is relevant: {'binary_score': 'yes'}
Answer Grading Result: {'binary_score': 'yes', 'explanation': 'The student answer provided information on types of agent memory, which includes: 1) Short-term memory, 2) Long-term memory, and 3) Working memory.'}


In [75]:
# Example usage
process_question("what is open ai Swarm?")

Document not relevant, performing web search...
Search Results: ['I can’t do that. I can provide general information about OpenAI Swarm. Would that help?']
Answer Grading Result: {'binary_score': 'yes', 'explanation': 'OpenAI Swarm is a software framework for building and training artificial intelligence models that can learn from large datasets and adapt to new situations. It was developed by OpenAI, a research organization founded by Elon Musk, and is designed to enable the creation of complex AI systems that can operate in real-world environments.'}


In [105]:
# Combined process_question function to use all agents
def process_question(question):
    """Process the question and return the final answer."""
    # Retrieve documents using retriever
    docs = retriever.invoke(question)
    doc_txt = docs[1].page_content if docs else None

    if doc_txt:
        # Use Retrieval Grader Agent to grade the retrieved document
        grade_result = grade_document(doc_txt, question)
        if grade_result.get("binary_score") == "yes":
            print(f"Retrieved document is relevant: {grade_result}")
        else:
            # If document is not relevant, use Web Search Agent
            print("Document not relevant, performing web search...")
            search_results = search_web(question)
            print(f"Search Results: {search_results}")
            doc_txt = "\n".join(search_results)
    else:
        # If no documents are retrieved, perform web search
        print("No documents retrieved, performing web search...")
        search_results = search_web(question)
        print(f"Search Results: {search_results}")
        doc_txt = "\n".join(search_results)

    # Use Hallucination Grader Agent before generating the answer
    if doc_txt:
        hallucination_grade_result = grade_hallucination(doc_txt, answer)
        if hallucination_grade_result.get("binary_score") == "no":
            print(f"Hallucination detected: {hallucination_grade_result}")
            return "The generated answer contains hallucinated information. Please verify the details."

    # Use Answer Grader Agent to grade a sample answer (for demonstration)
    sample_answer = "This is a sample answer based on the retrieved or searched information."
    answer_grade_result = grade_answer(question, sample_answer)
    print(f"Answer Grading Result: {answer_grade_result}")
# Example usage
process_question("What are the types of agent memory?")

Retrieved document is relevant: {'binary_score': 'yes'}
Answer Grading Result: {'binary_score': 'yes', 'explanation': 'The student answer provided information on types of agent memory, which includes: 1) Short-term memory, 2) Long-term memory, and 3) Working memory.'}


In [93]:
# Combined process_question function to use all agents
def process_question(question):
    """Process the question and return the final answer."""
    # Retrieve documents using retriever
    docs = retriever.invoke(question)
    doc_txt = docs[1].page_content if docs else None

    if doc_txt:
        # Use Retrieval Grader Agent to grade the retrieved document
        grade_result = grade_document(doc_txt, question)
        if grade_result.get("binary_score") == "yes":
            print(f"Retrieved document is relevant: {grade_result}")
        else:
            # If document is not relevant, use Web Search Agent
            print("Document not relevant, performing web search...")
            search_results = search_web(question)
            print(f"Search Results: {search_results}")
            doc_txt = "\n".join(search_results)
    else:
        # If no documents are retrieved, perform web search
        print("No documents retrieved, performing web search...")
        search_results = search_web(question)
        print(f"Search Results: {search_results}")
        doc_txt = "\n".join(search_results)

    # Use Hallucination Grader Agent before generating the answer
    if doc_txt:
        hallucination_grade_result = grade_hallucination(doc_txt, answer)
        if hallucination_grade_result.get("binary_score") == "no":
            print(f"Hallucination detected: {hallucination_grade_result}")
            return "The generated answer contains hallucinated information. Please verify the details."

    # Use Answer Grader Agent to grade a sample answer (for demonstration)
    sample_answer = "This is a sample answer based on the retrieved or searched information."
    answer_grade_result = grade_answer(question, sample_answer)
    print(f"Answer Grading Result: {answer_grade_result}")

# Example usage
process_question("who is indrajit kar?")


Document not relevant, performing web search...
Search Results: ['Indrajit Kar. Head of AI at Siemens Advanta. Product and AI Leader responsible for influencing CXOs & organizations with Data & algorithm-driven decision making, ML tech leadership, Data products development, solution design and roadmap. He owns the end to end client engagements as well as vendors management for emerging technologies that deal', "Indrajit Kar, a master's graduate in Computational Biology from Bengaluru, also holds a Bachelor's in Science from the same institution with more than two decades of experience in AI and ML. He is an experienced intrapreneur, having built AI teams at Siemens, Accenture, IBM, and Infinite Data Systems.", "Indrajit Kar, a master's graduate in Computational Biology from Bengaluru, also holds a Bachelor's in Science from the same institution with more than two decades of experience in AI and ML. He is an experienced intrapreneur, having built AI teams at Siemens, Accenture, IBM, and

'The generated answer contains hallucinated information. Please verify the details.'

In [149]:
# Combined process_question function to use all agents
def process_question(question):
    """Process the question and return the final answer."""
    # Retrieve documents using retriever
    docs = retriever.invoke(question)
    doc_txt = docs[1].page_content if docs else None

    if doc_txt:
        # Use Retrieval Grader Agent to grade the retrieved document
        grade_result = grade_document(doc_txt, question)
        if grade_result.get("binary_score") == "yes":
            print(f"Retrieved document is relevant: {grade_result}")
        else:
            # If document is not relevant, use Web Search Agent
            print("Document not relevant, performing web search...")
            search_results = search_web(question)
            print(f"Search Results: {search_results}")
            doc_txt = "\n".join(search_results)
    else:
        # If no documents are retrieved, perform web search
        print("No documents retrieved, performing web search...")
        search_results = search_web(question)
        print(f"Search Results: {search_results}")
        doc_txt = "\n".join(search_results)

    # Use Hallucination Grader Agent before generating the answer
    if doc_txt:
        hallucination_grade_result = grade_hallucination(doc_txt, answer)
        if hallucination_grade_result.get("binary_score") == "no":
            print(f"Hallucination detected: {hallucination_grade_result}")
            return "The generated answer contains hallucinated information. Please verify the details."

    # Use Answer Grader Agent to grade a sample answer (for demonstration)
    sample_answer = "This is a sample answer based on the retrieved or searched information."
    answer_grade_result = grade_answer(question, sample_answer)
    print(f"Answer Grading Result: {answer_grade_result}")



In [151]:
# Example usage
process_question("what is Open AI swarm?")

Document not relevant, performing web search...
Search Results: ['OpenAI Swarm is an open-source framework designed to simplify the creation and coordination of AI agents, focusing on agent coordination and execution through agents and handoffs.', "Swarm is OpenAI's innovative approach to creating a flexible and efficient framework for orchestrating multiple AI agents. Described as an ergonomic, lightweight, multi-agent orchestration framework , Swarm is currently in its experimental phase, aiming to explore user-friendly interfaces for managing complex multi-agent systems.", "OpenAI has announced and launched a new AI product called Swarm that showcases advances encompassing agentic AI. This is the future of AI. Here's what you need to know.", "OpenAI unveiled a new open-source framework last week designed to simplify the development and management of multi-agent AI systems that can collaborate autonomously to perform tasks. Dubbed 'Swarm,' the experimental release hints at a future w

'The generated answer contains hallucinated information. Please verify the details.'

In [157]:
# Combined process_question function to use all agents
def process_question(question):
    """Process the question and return the final answer."""
    # Retrieve documents using retriever
    docs = retriever.invoke(question)
    doc_txt = docs[1].page_content if docs else None

    if doc_txt:
        # Use Retrieval Grader Agent to grade the retrieved document
        grade_result = grade_document(doc_txt, question)
        if grade_result.get("binary_score") == "yes":
            print(f"Retrieved document is relevant: {grade_result}")
        else:
            # If document is not relevant, use Web Search Agent
            print("Document not relevant, performing web search...")
            search_results = search_web(question)
            print(f"Search Results: {search_results}")
            doc_txt = "\n".join(search_results)
    else:
        # If no documents are retrieved, perform web search
        print("No documents retrieved, performing web search...")
        search_results = search_web(question)
        print(f"Search Results: {search_results}")
        doc_txt = "\n".join(search_results)

    # Use Hallucination Grader Agent before generating the answer
    if doc_txt:
        hallucination_grade_result = grade_hallucination(doc_txt, answer)
        if hallucination_grade_result.get("binary_score") == "no":
            print(f"Hallucination detected: {hallucination_grade_result}")
            return "The generated answer contains hallucinated information. Please verify the details."

    # Use Answer Grader Agent to grade a sample answer (for demonstration)
    sample_answer = "This is a sample answer based on the retrieved or searched information."
    answer_grade_result = grade_answer(question, sample_answer)
    print(f"Answer Grading Result: {answer_grade_result}")



In [159]:
# Example usage
process_question("Decribe Genetic Algorithms?")

Document not relevant, performing web search...
Search Results: ['CiteSeerX - Genetic Algorithms (Oct. 04, 2024) genetic algorithm, in artificial intelligence, a type of evolutionary computer algorithm in which symbols (often called "genes" or "chromosomes") representing possible solutions are "bred.". This "breeding" of symbols typically includes the use of a mechanism analogous to the', 'A genetic algorithm is an optimisation or search algorithm that works essentially by mimicking the process of evolution. Genetic Algorithms are something Computer Science learnt from nature. For a diversion, let us first take a look at how evolution works as proposed by Charles Darwin in his The Origin of Species. Traits: Living Creatures consist of data about them encoded in their genetic', 'Data Structures and Algorithms\nML & Data Science\nWeb Development\nLanguages\nInterview Corner\nCS Subjects\nJobs\nPractice\nContests\nGenetic Algorithms\nGenetic Algorithms(GAs) are adaptive heuristic search a

'The generated answer contains hallucinated information. Please verify the details.'

In [165]:
# Example usage
process_question("what is handoff in agentic RAG?")

Document not relevant, performing web search...
Search Results: ['Agentic RAG represents an advanced AI system where autonomous agents leverage RAG techniques to enhance decision-making and response generation. Unlike traditional RAG models, which rely on user input to trigger actions, Agentic RAG systems operate proactively. These agents autonomously seek out relevant information, analyze it, and generate', 'In Native RAG the user is fed into the RAG pipeline which does retrieval, reranking, synthesis and generates a response. Agentic RAG is an agent based approach to perform question answering over…', 'Agentic RAG represents a significant evolution in information retrieval, introducing reasoning and intent recognition to enhance the accuracy and relevance of responses. By leveraging specialized', 'Agentic RAG: turbocharge your RAG with query reformulation and self-query! 🚀. Authored by: Aymeric Roucher This tutorial is advanced. You should have notions from this other cookbook first!

'The generated answer contains hallucinated information. Please verify the details.'